In [1]:
import pandas as pd
import numpy as np

# Load the data and generate returns

In [2]:
df = pd.read_excel('capm.xls', index_col = 0)
df.head()

,SANDP,FORD,GE,MICROSOFT,ORACLE,USTB3M
Date,,,,,,
2002-01-01,1130.20,12.57,25.70,27.33,16.64,1.73
2002-02-01,1106.73,12.23,26.75,25.02,16.03,1.75
2002-03-01,1147.39,13.55,25.99,25.87,12.34,1.77
2002-04-01,1076.92,13.15,21.92,22.41,9.68,1.78
2002-05-01,1067.14,14.59,21.64,21.84,7.64,1.77


In [3]:
assets = ['FORD', 'GE', 'MICROSOFT', 'ORACLE']

In [4]:
returns = pd.DataFrame()
returns['RM'] = np.log(df.SANDP/df.SANDP.shift(1))
returns['RF'] = df.USTB3M / 100 / 12
for asset in assets:
    returns[asset] = np.log(df[asset]/df[asset].shift(1))
returns.head()

,RM,RF,FORD,GE,MICROSOFT,ORACLE
Date,,,,,,
2002-01-01,NaN,0.001442,NaN,NaN,NaN,NaN
2002-02-01,-0.020985,0.001458,-0.027421,0.040043,-0.088309,-0.037347
2002-03-01,0.036080,0.001475,0.102495,-0.028823,0.033408,-0.261616
2002-04-01,-0.063385,0.001483,-0.029965,-0.170312,-0.143577,-0.242784
2002-05-01,-0.009123,0.001475,0.103915,-0.012856,-0.025764,-0.236664


In [5]:
returns['RM_e'] = returns.RM - returns.RF
assets_e = []
for asset in assets:
    assets_e.append(asset+'_e')
for i in range(len(assets)):
    returns[assets_e[i]] = returns[assets[i]] - returns.RF
returns.head()

,RM,RF,FORD,GE,MICROSOFT,ORACLE,RM_e,FORD_e,GE_e,MICROSOFT_e,ORACLE_e
Date,,,,,,,,,,,
2002-01-01,NaN,0.001442,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2002-02-01,-0.020985,0.001458,-0.027421,0.040043,-0.088309,-0.037347,-0.022443,-0.028879,0.038585,-0.089768,-0.038806
2002-03-01,0.036080,0.001475,0.102495,-0.028823,0.033408,-0.261616,0.034605,0.101020,-0.030298,0.031933,-0.263091
2002-04-01,-0.063385,0.001483,-0.029965,-0.170312,-0.143577,-0.242784,-0.064868,-0.031448,-0.171796,-0.145060,-0.244267
2002-05-01,-0.009123,0.001475,0.103915,-0.012856,-0.025764,-0.236664,-0.010598,0.102440,-0.014331,-0.027239,-0.238139


In [6]:
from patsy import dmatrices
import statsmodels.api as sm

# General Electric

In [7]:
y, X = dmatrices('GE_e ~ RM_e', data=returns, return_type='dataframe')
print(len(X))
X.head()

193


,Intercept,RM_e
Date,,
2002-02-01,1.0,-0.022443
2002-03-01,1.0,0.034605
2002-04-01,1.0,-0.064868
2002-05-01,1.0,-0.010598
2002-06-01,1.0,-0.076673


In [8]:
model = sm.OLS(y, X)    # Describe model
results = model.fit()       # Fit model
print(results.summary())   # Summarize model

                            OLS Regression Results                            
Dep. Variable:                   GE_e   R-squared:                       0.487
Model:                            OLS   Adj. R-squared:                  0.485
Method:                 Least Squares   F-statistic:                     181.5
Date:                Thu, 26 Sep 2019   Prob (F-statistic):           1.61e-29
Time:                        18:22:54   Log-Likelihood:                 286.24
No. Observations:                 193   AIC:                            -568.5
Df Residuals:                     191   BIC:                            -561.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0087      0.004     -2.179      0.0

In [9]:
sm.stats.diagnostic.het_white(results.resid, X, retres=False)

(5.854350372663208,
 0.053548088071907494,
 2.9718205392991686,
 0.053595776034563364)

In [16]:
sm.stats.diagnostic.acorr_breusch_godfrey(results, nlags=12, store=False)

(30.781971660773408,
 0.0021265131903923907,
 2.8305387219128484,
 0.0014077236820501278)

In [18]:
sm.stats.diagnostic.breaks_cusumolsresid(results.resid, ddof=1)

(1.2664477690527798, 0.0808872643934329, [(1, 1.63), (5, 1.36), (10, 1.22)])

# Ford

In [19]:
y, X = dmatrices('FORD_e ~ RM_e', data=returns, return_type='dataframe')
print(len(X))
X.head()

193


,Intercept,RM_e
Date,,
2002-02-01,1.0,-0.022443
2002-03-01,1.0,0.034605
2002-04-01,1.0,-0.064868
2002-05-01,1.0,-0.010598
2002-06-01,1.0,-0.076673


In [20]:
model = sm.OLS(y, X)    # Describe model
results = model.fit()       # Fit model
print(results.summary())   # Summarize model

                            OLS Regression Results                            
Dep. Variable:                 FORD_e   R-squared:                       0.339
Model:                            OLS   Adj. R-squared:                  0.335
Method:                 Least Squares   F-statistic:                     97.93
Date:                Thu, 26 Sep 2019   Prob (F-statistic):           6.68e-19
Time:                        18:37:17   Log-Likelihood:                 153.86
No. Observations:                 193   AIC:                            -303.7
Df Residuals:                     191   BIC:                            -297.2
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0085      0.008     -1.079      0.2

In [21]:
sm.stats.diagnostic.het_white(results.resid, X, retres=False)

(66.56994343973969,
 3.5036478725856526e-15,
 50.0208953379768,
 3.530508047109268e-18)

In [22]:
sm.stats.diagnostic.acorr_breusch_godfrey(results, nlags=12, store=False)

(44.407242833866874,
 1.3014591340558058e-05,
 4.457875683657406,
 3.032250506051561e-06)

In [23]:
sm.stats.diagnostic.breaks_cusumolsresid(results.resid, ddof=1)

(0.7777074769701744, 0.5808002843028807, [(1, 1.63), (5, 1.36), (10, 1.22)])

# MICROSOFT

In [24]:
y, X = dmatrices('MICROSOFT_e ~ RM_e', data=returns, return_type='dataframe')
print(len(X))
X.head()

193


,Intercept,RM_e
Date,,
2002-02-01,1.0,-0.022443
2002-03-01,1.0,0.034605
2002-04-01,1.0,-0.064868
2002-05-01,1.0,-0.010598
2002-06-01,1.0,-0.076673


In [25]:
model = sm.OLS(y, X)    # Describe model
results = model.fit()       # Fit model
print(results.summary())   # Summarize model

                            OLS Regression Results                            
Dep. Variable:            MICROSOFT_e   R-squared:                       0.383
Model:                            OLS   Adj. R-squared:                  0.380
Method:                 Least Squares   F-statistic:                     118.7
Date:                Thu, 26 Sep 2019   Prob (F-statistic):           8.22e-22
Time:                        18:41:27   Log-Likelihood:                 293.11
No. Observations:                 193   AIC:                            -582.2
Df Residuals:                     191   BIC:                            -575.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0018      0.004      0.468      0.6

In [26]:
sm.stats.diagnostic.het_white(results.resid, X, retres=False)

(0.30813693365085826,
 0.8572133283269234,
 0.15191616413378325,
 0.8591645395340689)

In [27]:
sm.stats.diagnostic.acorr_breusch_godfrey(results, nlags=12, store=False)

(16.958029924608383,
 0.15118291691979086,
 1.4369146153068517,
 0.15279880084548358)

In [28]:
sm.stats.diagnostic.breaks_cusumolsresid(results.resid, ddof=1)

(0.7122012476475021, 0.6908395209006682, [(1, 1.63), (5, 1.36), (10, 1.22)])

# ORACLE

In [29]:
y, X = dmatrices('ORACLE_e ~ RM_e', data=returns, return_type='dataframe')
print(len(X))
X.head()

193


,Intercept,RM_e
Date,,
2002-02-01,1.0,-0.022443
2002-03-01,1.0,0.034605
2002-04-01,1.0,-0.064868
2002-05-01,1.0,-0.010598
2002-06-01,1.0,-0.076673


In [30]:
model = sm.OLS(y, X)    # Describe model
results = model.fit()       # Fit model
print(results.summary())   # Summarize model

                            OLS Regression Results                            
Dep. Variable:               ORACLE_e   R-squared:                       0.331
Model:                            OLS   Adj. R-squared:                  0.328
Method:                 Least Squares   F-statistic:                     94.58
Date:                Thu, 26 Sep 2019   Prob (F-statistic):           2.06e-18
Time:                        18:45:06   Log-Likelihood:                 257.29
No. Observations:                 193   AIC:                            -510.6
Df Residuals:                     191   BIC:                            -504.0
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0009      0.005      0.198      0.8

In [31]:
sm.stats.diagnostic.het_white(results.resid, X, retres=False)

(1.7563753751070266,
 0.4155353094990406,
 0.8724769830232981,
 0.4195825692749361)

In [32]:
sm.stats.diagnostic.acorr_breusch_godfrey(results, nlags=12, store=False)

(15.253942448714326,
 0.22784199190009155,
 1.2801295173274745,
 0.23365175593980686)

In [33]:
sm.stats.diagnostic.breaks_cusumolsresid(results.resid, ddof=1)

(0.8092177592718804, 0.5292205212501961, [(1, 1.63), (5, 1.36), (10, 1.22)])